# **Recomendação para jogos digitais - Parte 2: Modelo** 🎮 

José Henrique de Oliveira<br>
[Portfólio](https://joseportfolio.notion.site/Portf-lio-do-Z-308350bb4f4544adb87da2a1a83b8f78) | [LinkedIn](https://www.linkedin.com/in/jholiveira94/)

---

# **Introdução**

A indústria de jogos digitais entrega grande volume de produções todos os anos e, como qualquer outro setor, necessita que seus produtos **desempenhem bem no mercado**.<br>
Portanto, **determinar quais nichos de jogos e clientes** trarão tal resultado é essencial para a continuidade dos projetos de uma empresa desse ramo.

Com esse contexto em mente, durante este estudo serão analisadas as vendas de jogos digitais com objetivo de encontrar padrões que auxiliem nessa tomada de decisão tão impactante.<br>
O conjunto de dados escolhido para conduzir essa investigação pode ser encontrado na plataforma Kaggle: [**Video Game Sales**](https://www.kaggle.com/datasets/gregorut/videogamesales/data)

Esse estudo terá como base a metodologia proposta no **CRISP-DM**:
1. Entendendo o negócio
2. Entendendo os dados
3. Preparação dos dados
4. Modelgem
5. Validação
6. Conclusões do estudo

Nesse notebook, serão continuadas as etapas do **CRISP-DM**, contemplado **parte da etapa 3**, agora com foco na **modelagem** de machine learning, **até a etapa 6**.

Em notebook anterior, foi trabalhado da **etapa 1 até parte da 3**, pois o foco foi a **EDA - Análise Exploratória dos Dados**. <br>
Para mais detalhes do estudo das variáveis do conjunto de dados selecionado, olhar o documento **Parte 01 - EDA.ipynb**.

### **Problemas de negócio**

Ao longo desse trabalho, será construído um **modelo de recomendação de lançamento de jogos por região** baseado nos padrões de vendas observados nos dados.<br>

##### **Objetivo**

<div style="background-color: #6a0dad; color: white; padding: 10px; border-radius: 5px;">
<b>Recomendação de gêneros populares por região</b> <br>
Quais gêneros de jogos devem ser lançados em uma região específica para maximizar as vendas?
</div>

Para aprofundar o estudo, foram respondidas questões mais objetivas durante a exploração dos dados, que se encontra no arquivo **Parte 01 - EDA.ipynb**.<br>

---

# **1. Preparações**

### **1.1 Bibliotecas e programações iniciais**

In [264]:
# Manipulação dos dados
import pandas as pd
import numpy as np

# Vizualização dos dados

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix

!pip install xgboost
from xgboost import XGBClassifier

# Filtro de avisos.
import warnings
warnings.filterwarnings('ignore')

---

# **2. Organização dos dados para modelagem**

In [211]:
# Carregamento dos dados
df = pd.read_csv('DataSet_vgsales.csv')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


Nesse conjunto de dados, há os seguintes atributos:

- Rank - Classificação geral das vendas
- Name - Nome do jogo
- Platform - Platforma de lançamento do jogo (PC,PS4, etc.)
- Year - Ano de lançamento do jogo
- Genre - Gênero do jogo
- Publisher - Empresa responsável pela publicação do jogo
- NA_Sales - Vendas na América do Norte (em milhões)
- EU_Sales - Vendas na Europa (em milhões)
- JP_Sales - Vendas no Japão (em milhões)
- Other_Sales - Vendas em outras regiões (em milhões)
- Global_Sales - Total mundial de vendas

In [212]:
''' 
Remoção de colunas que não serão usadas para construção do modelo.
Como feito no documento Parte 01 - EDA, as colunas serão traduzidas.
Há valores nulos no dataframe, como visto na exploração da Parte 1. As linhas com nulos serão removidas.
A coluna Ano precisa de tratamento na tipagem para int.
'''

df = df.drop(['Rank', 'Name', 'Platform'], axis = 1)
df.columns = ['Ano', 'Genero', 'Publisher', 'Vendas_AN', 'Vendas_EU', 'Vendas_JP', 'Vendas_Outras', 'Vendas_Globais']
df = df.dropna()
df['Ano'] = df.Ano.astype(int)
df = df.reset_index(drop = True)

# Verificando as alterações
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16291 entries, 0 to 16290
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ano             16291 non-null  int32  
 1   Genero          16291 non-null  object 
 2   Publisher       16291 non-null  object 
 3   Vendas_AN       16291 non-null  float64
 4   Vendas_EU       16291 non-null  float64
 5   Vendas_JP       16291 non-null  float64
 6   Vendas_Outras   16291 non-null  float64
 7   Vendas_Globais  16291 non-null  float64
dtypes: float64(5), int32(1), object(2)
memory usage: 954.7+ KB


### **2.1 Criação de novas features**

Nessa seção, serão definidas novas variáveis a partir dos dados selecionados.

Visando a interpretação do ano de lançamento dos jogos e do histórico de performance de cada gênero, será construída a feature "Tempo_Lançamento":
- Variável categórica.
- Classificação de cada jogo com base em seu ano de lançamento, isto é, uma espécie de fixa etária:
    - Atual: lançamento nos últimos 4 anos.
    - Recente: lançamento entre 4 e 10 anos.
    - Antigo: lançamento há mais de 10 anos.

In [213]:
''' 
Construção da feature "Tempo_Lançamento"
'''
def definir_faixa_etaria(ano):
    if ano <= (df["Ano"].max() - 10):
        return 'Antigo'
    elif ano <= (df["Ano"].max() - 4):
        return 'Recente'
    else:
        return 'Atual'

# Aplicando a função ao DataFrame
df['Tempo_Lançamento'] = df['Ano'].apply(definir_faixa_etaria)

'''  
Com a construção dessa nova coluna, a variável Ano torna-se redundante, portanto será retirada.
'''
df = df.drop('Ano', axis = 1)

Visando a interpretação da relevância das publishers em cada região, serão criadas colunas "Popularidade_[região]":
- Variável numérica.
- Cada publisher terá sua popularidade medida com base na média de venda em cada região (AN, EU, JP e Outras).
- Valores em milhões.

In [214]:
''' 
Construção das features "Popularidade_[região]"
'''

lista_popularidade = ['Popularidade_AN','Popularidade_EU', 'Popularidade_JP', 'Popularidade_Outras' ]

for i, regiao in enumerate(['Vendas_AN', 'Vendas_EU', 'Vendas_JP', 'Vendas_Outras']):
    df[lista_popularidade[i]] = df.groupby('Publisher')[regiao].transform('mean').round(2)

Considerando o objetivo desse projeto, será criada uma nova feature que indique a região com destaque de vendas de cada jogo: Regiao_Mais_Vendas
- Variável categórica.
- Receberá o nome da região em que obteve o maior valor em vendas: América do Norte, Europa, Japão ou Outras regiões.

Essa feature será a **variável foco** dos modelos de machine learning treinados.

In [215]:
''' 
Construção das features "Regiao_Mais_Vendas".
Primeiramente a coluna será criada com um valor qualquer que após será alterado conforme código.
'''
df['Regiao_Foco'] = 0
dic_regioes = {'Vendas_AN': 'América do Norte', 'Vendas_EU': 'Europa', 'Vendas_JP': 'Japão', 'Vendas_Outras': 'Outras regiões'}

for i in df.index:
    valor_max = df.iloc[i][['Vendas_AN', 'Vendas_EU', 'Vendas_JP', 'Vendas_Outras']].max()
    lista_maximos = []

    for r in ['Vendas_AN', 'Vendas_EU', 'Vendas_JP', 'Vendas_Outras']:
        if df[r][i] == valor_max:
            lista_maximos.append(r)
    
    lista_renomeada = [dic_regioes.get(elemento) for elemento in lista_maximos]
    df['Regiao_Foco'][i] = lista_renomeada[0]

In [216]:
# Visualizando o dataframe
df.head()

,Genero,Publisher,Vendas_AN,Vendas_EU,Vendas_JP,Vendas_Outras,Vendas_Globais,Tempo_Lançamento,Popularidade_AN,Popularidade_EU,Popularidade_JP,Popularidade_Outras,Regiao_Foco
0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Antigo,1.17,0.6,0.65,0.14,América do Norte
1,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Antigo,1.17,0.6,0.65,0.14,América do Norte
2,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Antigo,1.17,0.6,0.65,0.14,América do Norte
3,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Antigo,1.17,0.6,0.65,0.14,América do Norte
4,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Antigo,1.17,0.6,0.65,0.14,América do Norte


### **2.2 Tratamento das variáveis categóricas**

Para trabalhar com os modelos de machine learning selecionados, será necessário transformar os dados categóricos em numéricos.

Considerando a quantidade de publishers presentes nesse conjunto de dados e que a maioria causa pouco impacto no mercado, como visto na EDA, será realizada seleção desses dados:
- Será calculada a mediana das popularidades nas quatro regiões.
    - Foi escolhido usar a mediana visto que pode haver discrepância entre as popularidades de uma publisher e, nesses casos, ela não sofre grande impacto quanto a média.
- Serão **selecionadas as dez publishers** com maior mediana de popularidade.

In [217]:
df_mediana_popularidade = df[['Publisher', 'Popularidade_AN', 'Popularidade_EU', 'Popularidade_JP', 'Popularidade_Outras']].drop_duplicates().reset_index(drop = True)
df_mediana_popularidade['Mediana_Popularidade'] = 0

for i in df_mediana_popularidade.index:
    lista = []
    lista.append(df_mediana_popularidade['Popularidade_AN'][i])
    lista.append(df_mediana_popularidade['Popularidade_EU'][i])
    lista.append(df_mediana_popularidade['Popularidade_JP'][i])
    lista.append(df_mediana_popularidade['Popularidade_Outras'][i])
    
    df_mediana_popularidade['Mediana_Popularidade'][i] = np.median(lista)

df_mediana_popularidade = df_mediana_popularidade.sort_values(by = 'Mediana_Popularidade', ascending= False).reset_index(drop = True).head(10)

# Lista de publishers para filtrar dataframe
lista_top10_publishers = df_mediana_popularidade['Publisher']

# Filtrando o dataframe
df = df[df['Publisher'].isin(lista_top10_publishers)]
df.reset_index(inplace= True, drop = True)

In [218]:
# Verificando as mudanças
df.head()

,Genero,Publisher,Vendas_AN,Vendas_EU,Vendas_JP,Vendas_Outras,Vendas_Globais,Tempo_Lançamento,Popularidade_AN,Popularidade_EU,Popularidade_JP,Popularidade_Outras,Regiao_Foco
0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Antigo,1.17,0.6,0.65,0.14,América do Norte
1,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Antigo,1.17,0.6,0.65,0.14,América do Norte
2,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Antigo,1.17,0.6,0.65,0.14,América do Norte
3,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Antigo,1.17,0.6,0.65,0.14,América do Norte
4,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Antigo,1.17,0.6,0.65,0.14,América do Norte


Transformando features categóricas em numéricas para aplicação de modelos de machine learning.<br>
Para essa tarefa, será utilizado o método get_dummies(), da biblioteca Pandas.

> Esse método consiste em transformar cada um dos **valores** de uma variável categórica **em uma variável binária** (0 e 1, True e False,  etc.).<br>
> Dessa forma, os modelos de machine learning conseguem interpretar se ocorreu tal característica (1 ou True)  ou não (0 ou False).<br>
>
> **Por exemplo**:<br>
> O valor 'Sports' da variável Genero passa a ser uma coluna por si só. Quando o jogo for desse gênero, aparecerá 1 ou True nessa nova coluna 'Sports', do contrário, aparecerá 0 ou False.

Esse tratamento é necessário visto que os modelos de machine learning trabalham, em geral, com dados numéricos, portanto conseguem interpretar um valor '1', mas não conseguem processar se for 'Sports'.

In [279]:
# Aplicando Get Dummies
df_para_modelo = pd.get_dummies(df, columns=['Genero', 'Publisher', 'Tempo_Lançamento', 'Regiao_Foco'])

# Visualizando o resultado
df_para_modelo.head()

,Vendas_AN,Vendas_EU,Vendas_JP,Vendas_Outras,Vendas_Globais,Popularidade_AN,Popularidade_EU,Popularidade_JP,Popularidade_Outras,Genero_Action,...,Publisher_Red Orb,Publisher_RedOctane,Publisher_Sony Computer Entertainment Europe,Publisher_UEP Systems,Publisher_Valve,Tempo_Lançamento_Antigo,Tempo_Lançamento_Recente,Regiao_Foco_América do Norte,Regiao_Foco_Europa,Regiao_Foco_Japão
0,41.49,29.02,3.77,8.46,82.74,1.17,0.6,0.65,0.14,False,...,False,False,False,False,False,True,False,True,False,False
1,29.08,3.58,6.81,0.77,40.24,1.17,0.6,0.65,0.14,False,...,False,False,False,False,False,True,False,True,False,False
2,15.85,12.88,3.79,3.31,35.82,1.17,0.6,0.65,0.14,False,...,False,False,False,False,False,True,False,True,False,False
3,15.75,11.01,3.28,2.96,33.00,1.17,0.6,0.65,0.14,False,...,False,False,False,False,False,True,False,True,False,False
4,11.27,8.89,10.22,1.00,31.37,1.17,0.6,0.65,0.14,False,...,False,False,False,False,False,True,False,True,False,False


# **3. Modelagem**

Para atingir o objetivo de recomendar qual região será mais vantajosa para o lançamento de um jogo a partir de seu gênero, nesse estudo serão propostos modelos de **classificação**. <br>
Isto é, o modelo classificará qual região será mais interessante com base nas informações dos jogos já presentes no conjunto de dados.<br>
Essa classificação será interpretada como a sugestão de região a ser focada.

Serão testados inicialmente três modelos com o intuito de verificar qual apresenta melhores resultados:
- Decision tree
- Random Forest
- XGBoost

Após as primeiras avaliações, um dos modelos será escolhido para aprofundamento da modelagem.

**Preparação dos conjuntos de treino e teste.**<br>
Essa etapa é fundamental para o estudo dos modelos selecionados, pois ao treinar o modelo, ou ensinar, como ele deve proceder, é necessário verificar se suas previsões após aprendizado são razoáveis.<br>
Para fazer essa verificação, precisa-se de dados confiáveis e dos quais já se saiba as respostas.

Assim, a maior parte do conjunto de dados servirá de treino e uma parte menos de teste.<br>

In [228]:
x = df_para_modelo.drop(['Regiao_Foco_América do Norte', 'Regiao_Foco_Europa', 'Regiao_Foco_Japão'], axis=1)
y = df_para_modelo[['Regiao_Foco_América do Norte', 'Regiao_Foco_Europa', 'Regiao_Foco_Japão']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=13)

**Instanciando os três modelos.**<br>
Etapa de preparação. Os modelos da biblioteca SciKitLearn costumam necessitar desse processo, que consiste em determinar um objeto que guardará e funcionará com base nos comendas já estabelecidos na biblioteca.<br>
Após instanciado, basta entregar indicar os dados a serem trabalhados que o objeto saberá como proceder sem comandos adicionais.

In [231]:
# Decision Tree
modelo_dt = DecisionTreeClassifier()
modelo_dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [232]:
# Random Forest
modelo_rf = RandomForestClassifier()
modelo_rf.fit(x_train, y_train)

RandomForestClassifier()

In [233]:
# XGBoost
modelo_xgb = XGBClassifier()
modelo_xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)